In [1]:
import numpy as np
import torch as t
from mrftools import *

In [68]:
N = 25
lmb = 10
r_s = -5
r_h = -2


# uncontrolled dynamics
def phi_q(x1, x2):
    x1_x, x1_y = x1
    x2_x, x2_y = x2
    ind = ((x2_x == x1_x) and (x2_y == x1_y)) or \
            (x2_x == x1_x - 1) and (x2_y == x1_y) and (x1_x > 0) or \
            (x2_x == x1_x) and (x2_y == x1_y - 1) and (x1_y > 0) or \
            (x2_x == x1_x + 1) and (x2_y == x1_y) and (x1_x < np.sqrt(N)) or \
            (x2_x == x1_x) and (x2_y == x1_y + 1) and (x1_y < np.sqrt(N))
    return 1 if ind else 0

# from index in {0,...,N-1} to position (x,y) in {1,...,sqrt(N)}^2
def get_pos(index, N):
    index = index + 1 
    r = np.remainder(index, np.sqrt(N))
    if r == 0:
        x = np.sqrt(N)
        y = index // np.sqrt(N)
    else:
        x = r
        y = index // np.sqrt(N) + 1
    return x, y

# build uncontrolled dynamics matrix factor
phi_q_factor = t.zeros((N, N))
for i in range(N):
    for j in range(N):
        phi_q_factor[i, j] = phi_q(get_pos(i, N), get_pos(j, N))

mn = TorchMarkovNet(is_cuda=False, var_on=False)

factor = -float('inf')*t.ones(N)
factor[1] = 1
mn.set_unary_factor('x11', factor)

factor = -float('inf')*t.ones(N)
factor[14] = 1
mn.set_unary_factor('x12', factor)

mn.set_unary_factor('x21', t.ones(N))
mn.set_unary_factor('x31', t.ones(N))
mn.set_unary_factor('x22', t.ones(N))
mn.set_unary_factor('x32', t.ones(N))

r_hare = np.exp(-(r_h/lmb))
factor_h = t.ones(N)
factor_h[0] = r_hare
factor_h[4] = r_hare
factor_h[20] = r_hare
factor_h[24] = r_hare
mn.set_unary_factor('x41', factor_h)
mn.set_unary_factor('x42', factor_h)


mn.set_edge_factor(('x11', 'x21'), phi_q_factor)
mn.set_edge_factor(('x21', 'x31'), phi_q_factor)
mn.set_edge_factor(('x31', 'x41'), phi_q_factor)
mn.set_edge_factor(('x12', 'x22'), phi_q_factor)
mn.set_edge_factor(('x22', 'x32'), phi_q_factor)
mn.set_edge_factor(('x32', 'x42'), phi_q_factor)


r_stag = np.exp(-(r_s/lmb))
factor_s = t.ones((N, N))
factor_s[(12,12)] = r_stag
mn.set_edge_factor(('x41', 'x42'), factor_s)

bp = TorchMatrixBeliefPropagator(mn, is_cuda=False, var_on=True)

In [119]:
res_try_mat2 = t.tensor([1.75064451, 1.49727762, 1.38283992, 1.24779407, 0.28323888], dtype=t.float64)

In [120]:
ins_try_mat2 = t.tensor([[1.62434536, -2.50015019, -3.89192232, -0.98001738, -1.7067365],
                                 [-1.37513475, 1.47182113, -2.63786005, -float('inf'), -float('inf')],
                                 [-1.72311984, -float('inf'), 1.29355548, -float('inf'), -2.9843092],
                                 [-float('inf'), -float('inf'), -4.25291378, -float('inf'), -1.37131876],
                                 [-float('inf'), -float('inf'), -1.56686887, -float('inf'), -float('inf')]],
                                dtype=t.float64)

In [121]:
logsumexp(ins_try_mat2.numpy(), dim=0)

array([[ 1.70591277,  1.49048219,  1.37638705, -0.98001738, -0.72195103]])